<a href="https://colab.research.google.com/github/RonRichman/CredibilityTransformer/blob/main/The_Credibility_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credibility Transformer: A Novel Approach for Tabular Data Prediction
## Ronald Richman, Salvatore Scognamiglio, Mario V. Wüthrich

https://arxiv.org/pdf/2409.16653

Inspired by the success of Transformer architectures in natural language processing, the Credibility Transformer extends their application to tabular data. This method introduces a novel *credibility mechanism* that integrates prior information with observed data. By embedding tabular data into low-dimensional spaces, the model enables attention mechanisms similar to those in time-series data processing.

The core innovation lies in the use of a special "CLS token" representing prior information and its combination with learned data through a credibility-weighted averaging process. This structure enhances predictive performance, stabilizes training, and outperforms state-of-the-art models on tasks like motor insurance claims prediction.

This notebook implements a simplified version of the Credibility Transformer for tabular data, including steps for tokenization, embedding, and integration of the credibility mechanism. The goal is to demonstrate how the proposed method works, without too much complication of the code.

In [1]:
import os
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0


In [2]:
import keras
from keras.layers import Layer
import pandas as pd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
url='https://drive.google.com/file/d/1xIQJINshspnP2VshVrCVEl8blzOaxkMX/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
dat = pd.read_csv(url)
dat = pl.from_pandas(dat)
train = dat.filter(pl.col("set") == "train")
test = dat.filter(pl.col("set") == "test")
train = train.sample(n=train.shape[0], shuffle=True)
test = test.sample(n=test.shape[0], shuffle=True)

In [4]:
### check empirical proportions

dat.group_by("set").agg(
    pl.sum("ClaimNb") / pl.sum("Exposure"))

set,ClaimNb
str,f64
"""test""",0.07354
"""train""",0.073631


In [5]:
intercept_log = dat.group_by("set").agg(
    pl.sum("ClaimNb") / pl.sum("Exposure")).select("ClaimNb").to_numpy()[1]

In [6]:
var_names = dat.columns

cat_vars = [var_names[i-1] for i in [3,9,8,11]]

cont_vars = [var_names[i-1] for i in [4,5,6,7,10]]

In [7]:
### scale continuous and convert categorical to integers
def scale_data(col, method="min_max"):
    if method == "normalize":
        mean_val = col.mean()
        std_val = col.std()
        normalized = (col - mean_val) / std_val
        return {"mean": mean_val, "std": std_val, "normalized": normalized}

for col in cont_vars:
    stats = scale_data(train.select(col).to_series(), method="normalize")
    train = train.with_columns(stats["normalized"].alias(f"{col}_input"))
    test = test.with_columns(((pl.col(col) - stats["mean"]) / stats["std"]).alias(f"{col}_input"))

def cat_to_int(col):
    mapping = col.to_series().unique().sort().to_frame().with_columns(pl.arange(0, pl.count()).alias("col_name_input"))
    return {"map": mapping}

for col in cat_vars:
    maps = cat_to_int(train.select(col))["map"]
    maps = maps.rename({col: col, "col_name_input": f"{col}_input"})
    train = train.join(maps, on=col)
    test = test.join(maps, on=col)

train

<ipython-input-7-57e83b5f02a3>:15: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  mapping = col.to_series().unique().sort().to_frame().with_columns(pl.arange(0, pl.count()).alias("col_name_input"))


IDpol,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimTotal,ClaimNb,id,set,VehPower_input,VehAge_input,DrivAge_input,BonusMalus_input,Density_input,Area_input,VehGas_input,VehBrand_input,Region_input
f64,f64,str,i64,i64,i64,i64,str,str,f64,str,f64,i64,i64,str,f64,f64,f64,f64,f64,i64,i64,i64,i64
3.024116e6,0.2,"""D""",4,2,50,50,"""B12""","""Regular""",7.189922,"""R93""",0.0,0,367737,"""train""",-1.196788,-0.889183,0.318466,-0.624469,0.645439,3,1,3,20
4.044035e6,0.008197,"""F""",6,1,53,50,"""B12""","""Diesel""",10.035436,"""R11""",0.0,0,507668,"""train""",-0.22174,-1.065443,0.530678,-0.624469,2.165943,5,0,3,0
3.096508e6,0.35,"""A""",4,2,33,51,"""B3""","""Regular""",3.78419,"""R52""",0.0,0,405235,"""train""",-1.196788,-0.889183,-0.884068,-0.560505,-1.174418,0,1,7,11
2.118143e6,0.08,"""B""",8,0,46,50,"""B13""","""Diesel""",4.204693,"""R25""",0.0,0,258975,"""train""",0.753308,-1.241702,0.035517,-0.624469,-0.949722,1,0,4,5
164740.0,0.02,"""C""",9,3,32,80,"""B3""","""Regular""",5.204007,"""R82""",0.0,0,79216,"""train""",1.240832,-0.712923,-0.954805,1.294456,-0.415737,2,1,7,17
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1.040677e6,0.36,"""C""",8,11,50,50,"""B13""","""Regular""",5.370638,"""R82""",0.0,0,109682,"""train""",0.753308,0.697156,0.318466,-0.624469,-0.326698,2,1,4,17
140589.0,0.82,"""A""",7,2,38,50,"""B3""","""Regular""",3.89182,"""R24""",0.0,0,65898,"""train""",0.265784,-0.889183,-0.530381,-0.624469,-1.116906,0,1,7,4
2.098251e6,1.0,"""F""",6,8,53,50,"""B1""","""Diesel""",10.203592,"""R11""",0.0,0,247922,"""train""",-0.22174,0.168376,0.530678,-0.624469,2.255798,5,0,0,0


In [8]:
### get data in format for keras
def get_keras_data(dat):
    temp = {}
    for col in cat_vars + cont_vars:
        temp[f"{col}_input"] = dat.select(f"{col}_input").to_numpy()
    temp["Exposure"] = dat.select("Exposure").to_numpy()
    return temp

train_x = get_keras_data(train)
test_x = get_keras_data(test)

all_data = pl.concat([train, test])
all_x = get_keras_data(all_data)

train_y = train.select("ClaimNb").to_numpy()
test_y = test.select("ClaimNb").to_numpy()
train_x

{'Area_input': array([[3],
        [5],
        [0],
        ...,
        [5],
        [3],
        [2]]),
 'VehGas_input': array([[1],
        [0],
        [1],
        ...,
        [0],
        [0],
        [1]]),
 'VehBrand_input': array([[3],
        [3],
        [7],
        ...,
        [0],
        [3],
        [7]]),
 'Region_input': array([[20],
        [ 0],
        [11],
        ...,
        [ 0],
        [19],
        [17]]),
 'VehPower_input': array([[-1.1967877 ],
        [-0.22173974],
        [-1.1967877 ],
        ...,
        [-0.22173974],
        [ 0.26578424],
        [ 0.26578424]]),
 'VehAge_input': array([[-0.88918271],
        [-1.06544252],
        [-0.88918271],
        ...,
        [ 0.16837614],
        [-0.7129229 ],
        [-1.06544252]]),
 'DrivAge_input': array([[ 0.31846612],
        [ 0.53067802],
        [-0.88406799],
        ...,
        [ 0.53067802],
        [-0.03522038],
        [ 0.03551692]]),
 'BonusMalus_input': array([[-0.62446948],
     

In [9]:
class PositionalEmbedding(Layer):
    def __init__(self, embedding_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim

    def build(self, input_shape):
        self.pos_emb = self.add_weight(
            name='pos_emb',
            shape=(input_shape[1], self.embedding_dim),
            initializer='uniform',
            trainable=True
        )

    def call(self, inputs):
        batch_size = keras.ops.shape(inputs)[0]

        # Repeat self.pos_emb along the batch dimension to match the batch size
        expanded_pos_emb = keras.ops.broadcast_to(self.pos_emb, [batch_size, keras.ops.shape(self.pos_emb)[0], self.embedding_dim])

        return expanded_pos_emb

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.embedding_dim)


## add a single embedding of size embedding dim. This is a learned embedding that is the same for all rows.
### we will append this to the embeddings later, apply the transformer, and then strip this out as input into the LocalGLMnet's FFN.
### See the BERT paper for more details; arxiv link: https://arxiv.org/pdf/1810.04805.pdf

class ClsEmbedding(Layer):
    def __init__(self, embedding_dim, **kwargs):
        super(ClsEmbedding, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim

    def build(self, input_shape):
        self.cls_emb = self.add_weight(
            name='cls_emb',
            shape=(1, 1, self.embedding_dim),
            initializer='uniform',
            trainable=True
        )

    def call(self, inputs):
        batch_size = keras.ops.shape(inputs)[0]
        expanded_cls_emb = keras.ops.broadcast_to(self.cls_emb, [batch_size, 1, self.embedding_dim])
        return expanded_cls_emb

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1, self.embedding_dim)

### the aim of this next layer is to strip out the cls embedding, which now contains all the transformer information

class ClsTokenLayer(Layer):
    def __init__(self, **kwargs):
        super(ClsTokenLayer, self).__init__(**kwargs)

    def call(self, inputs):
        cls_token = inputs[:, -1, :]
        return cls_token

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1, input_shape[2])


class TransformerLayer(Layer):
    def __init__(self, q_dim=10, k_dim=10, v_dim=10, ffn_dim=32, dropout_rate=0.01, **kwargs):
        super(TransformerLayer, self).__init__(**kwargs)
        self.q_dim = q_dim
        self.k_dim = k_dim
        self.v_dim = v_dim
        self.ffn_dim = ffn_dim
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        self.layer_norm1 = keras.layers.TimeDistributed(keras.layers.LayerNormalization())
        self.q_dense = keras.layers.TimeDistributed(keras.layers.Dense(units=self.q_dim, activation="gelu"))
        self.k_dense = keras.layers.TimeDistributed(keras.layers.Dense(units=self.k_dim, activation="gelu"))
        self.v_dense = keras.layers.TimeDistributed(keras.layers.Dense(units=self.v_dim, activation="gelu"))
        self.attention = keras.layers.Attention(use_scale=True)
        self.dropout1 = keras.layers.Dropout(rate=self.dropout_rate)
        self.layer_norm2 = keras.layers.TimeDistributed(keras.layers.LayerNormalization())
        self.ffn_dense1 = keras.layers.TimeDistributed(keras.layers.Dense(units=self.ffn_dim, activation="gelu"))
        self.dropout2 = keras.layers.Dropout(rate=self.dropout_rate)
        self.ffn_dense2 = keras.layers.TimeDistributed(keras.layers.Dense(units=self.v_dim, activation="gelu"))
        self.dropout3 = keras.layers.Dropout(rate=self.dropout_rate)
        self.cls_token_layer = ClsTokenLayer()

    def call(self, inputs):
        embeds_norm = self.layer_norm1(inputs)
        q = self.q_dense(embeds_norm)
        k = self.k_dense(embeds_norm)
        v = self.v_dense(embeds_norm)
        attended = self.dropout1(self.attention([q, v, k]))
        skip1 = keras.layers.Add()([embeds_norm, attended])
        ffn = self.dropout3(self.ffn_dense2(self.dropout2(self.ffn_dense1(self.layer_norm2(skip1)))))
        skip2 = keras.layers.Add()([skip1, ffn])

        cls_token = self.cls_token_layer(v)
        cls_token = keras.ops.expand_dims(cls_token, axis=1)
        cls_token = self.layer_norm2(cls_token)
        cls_token = self.ffn_dense1(cls_token)
        cls_token = self.dropout2(cls_token)
        cls_token = self.ffn_dense2(cls_token)
        cls_token = keras.layers.Flatten()(cls_token)

        return [skip2, cls_token]

    def compute_output_shape(self, input_shape):
        return [input_shape, input_shape]

In [10]:
### define NN
keras.backend.clear_session()
input_list = []
embedding_list = []
cardinalities = {}
count = 0

### loop over categorical. for each, define an input layer and an output layer

### here we set embed dim to 2

for column in [f"{col}_input" for col in cat_vars]:
    count += 1
    cardinalities[column] = train[column].max()
    print(cardinalities[column])
    input_list.append(keras.layers.Input(shape=(1,), dtype="int32", name=column))
    embedding_list.append(keras.layers.Embedding(input_dim=cardinalities[column]+1, output_dim=5)(input_list[-1]))
    embedding_list[-1] = keras.layers.Flatten(name=f"{column}_embed")(embedding_list[-1])

### loop over continuous, for each, define an input layer and an output layer
### here we apply a small neural network to get the continuous variables into the same dimension as the categorical variables

for column in [f"{col}_input" for col in cont_vars]:
    count += 1
    input_list.append(keras.layers.Input(shape=(1,), dtype="float32", name=column))
    embedding_list.append(keras.layers.Dense(units=5)(input_list[-1]))
    embedding_list[-1] = keras.layers.Dense(units=5, activation="tanh")(embedding_list[-1])

input_list.append(keras.layers.Input(shape=(1,), dtype="float32", name="Exposure"))

embeds = keras.layers.Concatenate(axis=1)(embedding_list)
embeds = keras.layers.Reshape((9, 5))(embeds)

### here we add the positional embedding to the categorical embeddings; this is one embedding for each column that
### is the same for all the rows

positional_embedding_layer = PositionalEmbedding(embedding_dim=5)
positional_embedding_layer = positional_embedding_layer(embeds)
embeds = keras.layers.Concatenate()([embeds, positional_embedding_layer])

# Create an instance of the cls layer
cls_embedding_layer = ClsEmbedding(embedding_dim=10)

# get the cls embedding
cls_embed = cls_embedding_layer(embeds)
embeds = keras.layers.Concatenate(axis=1)([embeds, cls_embed])

### Custom Multi-Head Attention Transformer Layer

transformer_layer = TransformerLayer(q_dim=10, k_dim=10, v_dim=10, ffn_dim=40, dropout_rate=0.01)
transformer_output = transformer_layer(embeds)

class VectorSelectionLayer(keras.layers.Layer):
    def __init__(self, rate, apply_during_test=False, **kwargs):
        super().__init__(**kwargs)
        self.rate = rate
        self.apply_during_test = apply_during_test
        self.seed_generator = keras.random.SeedGenerator(seed=1337)

    def call(self, inputs, training=None):
        vector1, vector2 = inputs

        if training or self.apply_during_test:
            random_tensor = keras.random.uniform(shape=(keras.ops.shape(vector1)[0],), seed=self.seed_generator)
            selection_mask = keras.ops.cast(random_tensor < self.rate, dtype='float32')
            selection_mask = keras.ops.expand_dims(selection_mask, axis=-1)
            selected_vector = vector1 * selection_mask + vector2 * (1 - selection_mask)
        else:
            selected_vector = vector1

        return selected_vector

    def compute_output_shape(self, input_shape):
        return input_shape[0]

### strip out the cls embedding, which now contains all the transformer information

cls_token_layer = ClsTokenLayer()
cls_token = cls_token_layer(transformer_output[0])

# Create an instance of the VectorSelectionLayer
vector_selection_layer = VectorSelectionLayer(rate=0.9, apply_during_test=False)

# Apply the layer
transformer_output2 = vector_selection_layer([cls_token, transformer_output[1]])

### rest of the network
### we use a very simple FFN
initializer = keras.initializers.Constant(0)
initializer_bias = keras.initializers.Constant(np.log(intercept_log))

output_rescale = keras.layers.Dense(units=16, activation="gelu")(keras.layers.Flatten()(transformer_output2))
output_rescale = keras.layers.Dense(units=1, activation="exponential", kernel_initializer=initializer, bias_initializer= initializer_bias)(output_rescale)

output = keras.layers.Multiply(name="output")([output_rescale, input_list[-1]])

### define model

model = keras.Model(inputs=input_list, outputs=output)

5
1
10
21


In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Area_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ VehGas_input (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ VehBrand_input            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Region_input (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ VehPower_input            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ VehAge_input (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ DrivAge_input             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ BonusMalus_input          │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Density_input             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 5)           │             30 │ Area_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 1, 5)           │             10 │ VehGas_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 1, 5)           │             55 │ VehBrand_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 1, 5)           │            110 │ Region_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 5)              │             10 │ VehPower_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 5)              │             10 │ VehAge_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 5)              │             10 │ DrivAge_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 5)              │             10 │ BonusMalus_input[0][0] │
├──────────────────────

 Total params: 653 (2.55 KB)

 Trainable params: 653 (2.55 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
### see paper for source on beta_2

adamw = keras.optimizers.AdamW(learning_rate=0.001, weight_decay=0.01, beta_2 = 0.95)

model.compile(optimizer=adamw, loss="poisson")


In [13]:
model_write =keras.callbacks.ModelCheckpoint("frmtpl_cred_transformer.weights.h5", save_best_only=True, verbose=1, save_weights_only=True)
learn_rate = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.9, patience=20, verbose=1)
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)
# fit model.

fit = model.fit(x=train_x, y=train_y, batch_size=1024, epochs=250, callbacks=[model_write,learn_rate,early_stop],
                validation_split=0.1, verbose=1, shuffle=True)

Epoch 1/250
535/537 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1622
Epoch 1: val_loss improved from inf to 0.15362, saving model to frmtpl_cred_transformer.weights.h5
537/537 ━━━━━━━━━━━━━━━━━━━━ 36s 35ms/step - loss: 0.1622 - val_loss: 0.1536 - learning_rate: 0.0010
Epoch 2/250
535/537 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1585
Epoch 2: val_loss improved from 0.15362 to 0.15256, saving model to frmtpl_cred_transformer.weights.h5
537/537 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - loss: 0.1585 - val_loss: 0.1526 - learning_rate: 0.0010
Epoch 3/250
535/537 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1560
Epoch 3: val_loss improved from 0.15256 to 0.15213, saving model to frmtpl_cred_transformer.weights.h5
537/537 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - loss: 0.1560 - val_loss: 0.1521 - learning_rate: 0.0010
Epoch 4/250
535/537 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1555
Epoch 4: val_loss did not improve from 0.15213
537/537 ━━━━━━━━━━━━━━━━━━━━ 16s 30ms/step - loss: 0.1555 - val_loss:

In [14]:
model.load_weights("frmtpl_cred_transformer.weights.h5")

In [15]:
import numpy as np

train_pred = model.predict(train_x, batch_size=32000)
test_pred = model.predict(test_x, batch_size=32000)

20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [16]:
import polars as pl

# Ensure train_pred and test_pred are flattened to match the DataFrame structure
train_pred_flat = train_pred.flatten()
test_pred_flat = test_pred.flatten()

# Add the predictions to the DataFrames
train = train.with_columns(pl.Series("pred_NN", train_pred_flat))
test = test.with_columns(pl.Series("pred_NN", test_pred_flat))

### scoring
def score_model(train, test, pred_col):
    train_pred = train.select(pred_col).to_numpy()
    test_pred = test.select(pred_col).to_numpy()

    def poisson_deviance(true, pred):
        sum_pred = np.sum(pred)
        sum_true = np.sum(true)
        log_term = np.sum((true + 1e-15) * np.log((true + 1e-15) / (pred + 1e-15)))
        print("sum_pred:", sum_pred)
        print("sum_true:", sum_true)
        print("log_term:", log_term)
        return 2 * (sum_pred - sum_true + log_term) / len(true)

    pois_dev = poisson_deviance(train.select("ClaimNb").to_numpy(), train_pred)
    pois_dev_test = poisson_deviance(test.select("ClaimNb").to_numpy(), test_pred)
    results = {
        "pois_dev": pois_dev,
        "pois_dev_test": pois_dev_test,
        "model": pred_col
    }

    return {"results": results}

# Ensure you are passing the correct DataFrames
score_model(train, test, "pred_NN")

sum_pred: 22872.844
sum_true: 23738
log_term: 73029.56787943454
sum_pred: 2561.6936
sum_true: 2645
log_term: 8129.847907163831


{'results': {'pois_dev': 0.23652475272099766,
  'pois_dev_test': 0.23735760566007746,
  'model': 'pred_NN'}}